In [ ]:
# =============================
# 🧠 Protein Function Prediction - Improved Model
# =============================

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.callbacks import EarlyStopping
import itertools
from Bio import SeqIO
from sklearn.utils.class_weight import compute_class_weight
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder






fasta_path = r"C:\Users\imiss\Desktop\protein-function-prediction\data\Train\train_sequences.fasta"
tsv_path = r"C:\Users\imiss\Desktop\protein-function-prediction\data\Train\train_terms.tsv"






seq_records = list(SeqIO.parse(fasta_path, "fasta"))
seq_data = pd.DataFrame({
    "EntryID": [r.id for r in seq_records],
    "sequence": [str(r.seq) for r in seq_records]
})
seq_data["EntryID"] = seq_data["EntryID"].str.split("|").str[1].fillna(seq_data["EntryID"])


term_data = pd.read_csv(tsv_path, sep="\t")


df = term_data.merge(seq_data, on="EntryID", how="inner")
df = df.dropna(subset=["sequence"])
df = df[df["sequence"].str.len() > 0]
print("✅ Combined data shape:", df.shape)
print(df.head())



term_data = pd.read_csv(tsv_path, sep="\t")


df = term_data.merge(seq_data, on="EntryID", how="inner")
print("✅ Combined data shape:", df.shape)
print(df.head())


df = df.dropna(subset=["sequence"])
df = df[df["sequence"].str.len() > 0]
print("✅ Final dataset size:", len(df))
print(df.sample(5))





df.columns = df.columns.str.strip().str.lower()
df.rename(columns={"real_go": "term", "predicted_go": "sequence"}, inplace=True)

print("✅ Columns:", df.columns.tolist())
print("✅ Total rows:", len(df))


df = df.drop_duplicates(subset=["sequence"]).dropna(subset=["sequence", "term"])
print("✅ After cleanup:", df.shape)


top_terms = df["term"].value_counts().nlargest(50).index
df = df[df["term"].isin(top_terms)]


print("✅ Rows after expanding filter:", len(df))
print("✅ Unique terms remaining:", df["term"].nunique())







le = LabelEncoder()
y = le.fit_transform(df["term"])




aa_cols = list("ACDEFGHIKLMNPQRSTVWY")
X_df = pd.DataFrame(X[:, :20], columns=aa_cols)

corr = X_df.corr()

def feature_vector(seq):
    seq = seq.upper()
    L = len(seq)
    aa_counts  = [seq.count(a)/max(L,1) for a in aas]
    di_counts  = [seq.count(d)/max(L-1,1) for d in dipeptides]
    return aa_counts + di_counts

X = np.vstack(df["sequence"].apply(feature_vector))
print("✅ New feature shape:", X.shape)




scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)
print("✅ Split complete:", X_train.shape, X_test.shape)




es = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

model = Sequential([
    Dense(256, activation='relu', input_shape=(X.shape[1],)),
    Dropout(0.2),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(len(le.classes_), activation='softmax')
])

model.compile(
    optimizer=Adam(learning_rate=3e-4),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])



y_pred = model.predict(X_test)
y_pred_classes = y_pred.argmax(axis=1)


cm = confusion_matrix(y_test, y_pred_classes)
cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]  


print("Confusion matrix shape:", cm.shape)




cm = confusion_matrix(y_test, y_pred_classes)
cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]  



print("Confusion matrix shape:", cm.shape)





cw = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weight = dict(zip(np.unique(y_train), cw))

history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=100,
    batch_size=32,
    class_weight=class_weight,
    callbacks=[es],
    verbose=1
)


loss, acc = model.evaluate(X_test, y_test)
print(f"\n🧾 FINAL TEST ACCURACY: {acc*100:.2f}%")


import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay



subset_classes = le.classes_[:15]  

plt.figure(figsize=(12, 10))
sns.heatmap(
    cm_norm[:15, :15], 
    annot=True, fmt=".2f", cmap="mako", 
    xticklabels=subset_classes,
    yticklabels=subset_classes
)
plt.title("Protein Function Prediction – Normalized Confusion Matrix (Top 15 GO Terms)")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()


plt.figure(figsize=(6,1))
sns.heatmap([[acc*100]], annot=True, fmt=".2f", cmap="YlGnBu", cbar=False)
plt.title("🧾 Final Test Accuracy (%)")
plt.show()




plt.figure(figsize=(12, 10))
sns.heatmap(
    corr,
    annot=True,
    cmap="coolwarm",
    center=0,
    square=True,
    linewidths=0.5,
    cbar_kws={"shrink": 0.8}
)
plt.title("Correlation Heatmap of Amino Acid Composition", fontsize=14, pad=20)
plt.xticks(rotation=45, ha="right")
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()





plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.legend(); plt.title('Accuracy over Epochs')

plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.legend(); plt.title('Loss over Epochs')
plt.tight_layout(); plt.show()
